In [57]:
import pandas as pd
import yfinance as yf
import numpy as np 

In [58]:
portf = pd.read_csv("portfolio_holdings.csv")

cash_row = portf[portf['TICKER'] == 'USD']
portf = portf[portf['TICKER'] != 'USD']

portf.tail()

,TICKER,QUANTITY
32,VICI,112.0
33,VIS,15.0
34,VOX,87.0
35,VPU,19.0
36,YUM,17.0


In [59]:
# Fetch data for the tickers in the portfolio
tickers = portf['TICKER'].tolist()
data = yf.download(tickers, start="2019-01-01", end="2024-01-01", interval="1mo")['Adj Close']

# Calculate monthly returns
returns = data.pct_change().dropna()


[*********************100%***********************]  37 of 37 completed


In [60]:
# Calculate total investment in each stock
current_prices = data.iloc[-1]  # Last row gives the latest prices
portf['Investment'] = portf['QUANTITY'] * current_prices.values

# Add the cash row back for total investment
total_investment = portf['Investment'].sum() + cash_row['QUANTITY'].iloc[0]

# Calculate weights
portf['Weight'] = portf['Investment'] / total_investment

print(portf[['TICKER', 'Weight']])

# Add a weight of cash (cash has zero returns)
cash_weight = cash_row['QUANTITY'].iloc[0] / total_investment
print(f"Cash Weight: {cash_weight:.2%}")

   TICKER    Weight
0    ADBE  0.023706
1     ADM  0.012302
2    ATKR  0.013882
3     BBY  0.019046
4     BLK  0.018871
5     BRC  0.009222
6    CPAY  0.022459
7     CVI  0.006861
8     CVX  0.006888
9     DIS  0.003560
10    FDX  0.017733
11  GOOGL  0.034329
12    JBL  0.016166
13    JPM  0.019925
14   LDOS  0.000850
15    LMT  0.017660
16   LRCX  0.003597
17    MCK  0.014665
18    MET  0.022425
19    MOH  0.017228
20    MPC  0.005811
21    MYE  0.013374
22    OVV  0.006454
23   SCHW  0.009698
24    TEL  0.019843
25    VAW  0.019306
26    VCR  0.038398
27    VDC  0.042828
28    VDE  0.007217
29    VFH  0.046018
30    VGT  0.137517
31    VHT  0.088366
32   VICI  0.026848
33    VIS  0.025925
34    VOX  0.080507
35    VPU  0.020018
36    YUM  0.017392
Cash Weight: 9.31%


In [61]:
# Calculate weighted monthly returns for the portfolio
weighted_returns = (returns * portf.set_index('TICKER')['Weight']).sum(axis=1)

# Adjust weighted returns to include cash (cash return is zero)
weighted_returns = weighted_returns * (1 - cash_weight)

# Expected annual return
expected_return = weighted_returns.mean() * 12
print(f"Expected Annual Return: {expected_return:.2%}")

# Portfolio variance and standard deviation (risk)
portfolio_variance = np.dot(portf.set_index('TICKER')['Weight'].T, 
                            np.dot(returns.cov() * 12, 
                                   portf.set_index('TICKER')['Weight']))
portfolio_variance *= (1 - cash_weight)**2  # Adjust for cash weight
portfolio_std_dev = np.sqrt(portfolio_variance)
print(f"Portfolio Standard Deviation (Annualized): {portfolio_std_dev:.2%}")


Expected Annual Return: 14.53%
Portfolio Standard Deviation (Annualized): 16.69%


In [62]:
# Download S&P 500 (or another market index) data
market_data = yf.download('^GSPC', start="2019-01-01", end="2024-01-01", interval="1mo")['Adj Close']

# Calculate market monthly returns
market_returns = market_data.pct_change().dropna()


[*********************100%***********************]  1 of 1 completed


In [63]:
# Align both series and drop NaNs
aligned_portfolio_returns, aligned_market_returns = weighted_returns.align(market_returns, join='inner')

# Ensure no missing values remain after alignment
aligned_portfolio_returns = aligned_portfolio_returns.dropna()
aligned_market_returns = aligned_market_returns.dropna()

print(f"After cleaning: {len(aligned_portfolio_returns)} vs {len(aligned_market_returns)}")


After cleaning: 59 vs 59


In [64]:
# Ensure aligned_market_returns is a Series
if isinstance(aligned_market_returns, pd.DataFrame):
    aligned_market_returns = aligned_market_returns.squeeze()  # Convert single-column DataFrame to Series


In [65]:
cov_matrix = np.cov(aligned_portfolio_returns, aligned_market_returns)

# Calculate beta
beta = cov_matrix[0, 1] / cov_matrix[1, 1]
print(f"Portfolio Beta: {beta:.2f}")


Portfolio Beta: 0.89


In [66]:
# We need risk free frate for calculating the sharpe ratio, so we're just gonna use yfinance and fetch TNX. If you want perfectly accurate sharpe calculation you can use U.S. treasury data from the internet for that. 10-yr bonds.
# Fetch 10-Year Treasury Yield (^TNX)
risk_free_data = yf.download('^TNX', start="2019-01-01", end="2024-01-01", interval="1mo")

# Check the structure of risk_free_data
print(risk_free_data.tail())  # Verify the last few rows of data

# Ensure you get the last valid yield as a scalar
risk_free_rate = risk_free_data['Adj Close'].dropna().iloc[-1] / 100  # Ensure single value and convert
risk_free_rate


[*********************100%***********************]  1 of 1 completed

Price                     Adj Close  Close   High    Low   Open Volume
Ticker                         ^TNX   ^TNX   ^TNX   ^TNX   ^TNX   ^TNX
Date                                                                  
2023-08-01 00:00:00+00:00     4.093  4.093  4.362  3.957  4.001      0
2023-09-01 00:00:00+00:00     4.573  4.573  4.688  4.060  4.097      0
2023-10-01 00:00:00+00:00     4.875  4.875  4.997  4.532  4.631      0
2023-11-01 00:00:00+00:00     4.352  4.352  4.903  4.253  4.893      0
2023-12-01 00:00:00+00:00     3.866  3.866  4.348  3.785  4.320      0


Ticker
^TNX    0.03866
Name: 2023-12-01 00:00:00+00:00, dtype: float64

In [67]:
# Sharpe Ratio
sharpe_ratio = (expected_return - risk_free_rate)/portfolio_std_dev
sharpe_ratio

# Interpretation
# Sharpe Ratio > 1: Good (your portfolio provides solid risk-adjusted returns).
# Sharpe Ratio > 2: Very good.
# Sharpe Ratio > 3: Excellent.
# Sharpe Ratio < 1: Risk-adjusted returns are relatively low; may need to re-evaluate risk.

Ticker
^TNX    0.638868
Name: 2023-12-01 00:00:00+00:00, dtype: float64